<a href="https://colab.research.google.com/github/Caveman-cell/32602200088_Faridd/blob/main/coba1_2_buah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Klasifikasi gambar jeruk dan semangka

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Path ke dataset
dataset_path = "sample_data/dataset/"

# Memuat dataset
train_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Normalisasi gambar
normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

# Membangun model CNN sederhana
from tensorflow.keras import Input

model = models.Sequential([
    Input(shape=(224, 224, 3)),  # Menentukan bentuk input
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # 2 kelas: jeruk dan semangka
])

# Kompilasi model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Melatih model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

# Menyimpan model
model.save("model_jeruk_semangka.h5")


Found 6 files belonging to 2 classes.
Using 5 files for training.
Found 6 files belonging to 2 classes.
Using 1 files for validation.
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6000 - loss: 0.6678 - val_accuracy: 0.0000e+00 - val_loss: 10.4207
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6000 - loss: 4.5959 - val_accuracy: 1.0000 - val_loss: 2.0742e-05
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 610ms/step - accuracy: 0.4000 - loss: 6.1761 - val_accuracy: 1.0000 - val_loss: 0.0705
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 619ms/step - accuracy: 1.0000 - loss: 0.3201 - val_accuracy: 0.0000e+00 - val_loss: 2.7848
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step - accuracy: 0.6000 - loss: 0.7498 - val_accuracy: 0.0000e+00 - val_loss: 0.7461
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 604ms/step - accuracy: 1.0000 - loss: 0.0882 - val_accuracy: 1.0000 - val_loss: 0.0185
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step - accuracy: 1.0000 - loss: 0.2344 - val_accuracy:

Test 1 gambar

In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Memuat model
model = load_model("model_jeruk_semangka.h5")

# Path ke gambar untuk prediksi
img_path = "sample_data/dataset/semangka/smpl1.jpg"  # Ganti dengan path gambar yang ingin diprediksi

# Memuat dan memproses gambar
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0  # Normalisasi
img_array = np.expand_dims(img_array, axis=0)

# Prediksi
predictions = model.predict(img_array)
class_names = ['jeruk', 'semangka']  # Nama kelas sesuai urutan folder
predicted_class = class_names[np.argmax(predictions)]

print(f"Gambar: {img_path}")
print(f"Prediksi: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Gambar: sample_data/dataset/semangka/smpl1.jpg
Prediksi: semangka


insialisasi ulang pada optimizer

In [14]:
from tensorflow.keras.optimizers import Adam

# Inisialisasi ulang optimizer
optimizer = Adam(learning_rate=0.001)

# Kompilasi ulang model
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


More Epochs

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

# Callback Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Mengamati loss pada validasi
    patience=15,          # Menghentikan jika tidak ada peningkatan selama 3 epoch berturut-turut
    restore_best_weights=True  # Memulihkan bobot terbaik model
)

# Melatih model dengan lebih banyak epoch
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,  # Bisa lebih banyak dari 10
    callbacks=[early_stopping]  # Menggunakan Early Stopping
)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4000 - loss: 3.3710 - val_accuracy: 1.0000 - val_loss: 9.1237e-04
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 909ms/step - accuracy: 1.0000 - loss: 7.9602e-05 - val_accuracy: 1.0000 - val_loss: 0.2158
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step - accuracy: 1.0000 - loss: 6.8125e-04 - val_accuracy: 0.0000e+00 - val_loss: 1.1748
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 642ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 0.5623
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step - accuracy: 1.0000 - loss: 3.6926e-04 - val_accuracy: 1.0000 - val_loss: 0.0122
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step - accuracy: 1.0000 - loss: 2.6226e-06 - val_accuracy: 1.0000 - val_loss: 3.4451e-05
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 646ms/step - accuracy: 1.0000 - loss: 2.9087e-06 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 595ms/step - accuracy: 1.0000 - loss: 0.0082 -

Cek akurasi

In [18]:
# Evaluasi pada dataset validasi
loss, accuracy = model.evaluate(val_dataset)
print(f"Akurasi pada data validasi: {accuracy * 100:.2f}%")
print(f"Loss pada data validasi: {loss:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Akurasi pada data validasi: 100.00%
Loss pada data validasi: 0.0000


Test upload gambar

In [20]:
from google.colab import files
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Memuat model
model = load_model("model_jeruk_semangka.h5")

# Upload gambar dari komputer
uploaded = files.upload()

# Ambil nama file gambar yang diupload
img_path = next(iter(uploaded))  # Mengambil nama file pertama yang diupload

# Memuat gambar dan mengubah ukurannya ke 224x224 (ukuran input model)
img = image.load_img(img_path, target_size=(224, 224))

# Mengonversi gambar ke array dan melakukan normalisasi
img_array = image.img_to_array(img) / 255.0  # Normalisasi
img_array = np.expand_dims(img_array, axis=0)  # Tambahkan batch dimension

# Prediksi gambar
predictions = model.predict(img_array)

# Nama kelas
class_names = ['jeruk', 'semangka']

# Menampilkan hasil prediksi
predicted_class = class_names[np.argmax(predictions)]
print(f"Prediksi: {predicted_class}")


Saving smp2.jpg to smp2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Prediksi: jeruk
